In [1]:
from IPython.display import clear_output
import os

In [9]:
!pip install langchain-openai langchain langchain-community langchain-core
clear_output()

In [10]:
from langchain_openai import OpenAI #Clase base para interactuar con OpenAI (ver ant deprecado from langchain.llms import OpenAI)
from langchain.chains import LLMChain #Clase base para crear cadenas
from langchain.prompts import PromptTemplate #se utiliza para definir plantillas de prompts reutilizables, con variables dinámicas
from langchain.prompts import ChatPromptTemplate #clase diseñada para trabajar específicamente con modelos de chat en modo de conversación.

In [11]:
#importamos la api
with open("/content/api_key.txt") as archivo:
  apikey = archivo.read()
os.environ["OPENAI_API_KEY"] = apikey



---



## Modelo de cadena de texto

- Modelos antiguos Estos modelos más antiguos aceptan una cadena de texto como entrada y devuelven una cadena de texto como salida
- Modelos con capacidad de conversación ("Chat"): Estos modelos avanzados son diseñados específicamente para interacciones conversacionales

In [18]:
# Vamos a hacer una prueba mediante OpenAI usando un modelo antigu modo no conversacional soporte gpt-3.5-turbo-instruct

llm = OpenAI(temperature=0.5, model="gpt-3.5-turbo-instruct")

response = llm.invoke("¿Cual es la capital de Peru?")

print(response)



La capital de Perú es Lima.




---



## PromptTemplate

Diseñado para crear prompts dinámicos con variables, generalmente para tareas no conversacionales.

In [21]:
#Template
prompt_template =PromptTemplate.from_template("Describe el siguiente producto de una forma convincente y breve {product_name}")

#instancia
llm = OpenAI(temperature=0.5)

#cadena
chain = LLMChain(llm=llm, prompt=prompt_template)

#ejecutamos
input = {'product_name':'Botella de agua ecologica'}
print(chain.run(input))



La botella de agua ecológica es la opción perfecta para aquellos que buscan una alternativa sostenible y amigable con el medio ambiente. Fabricada con materiales reciclables y biodegradables, esta botella te permite hidratarte de forma responsable y sin generar residuos dañinos para nuestro planeta. Su diseño moderno y funcional la convierte en un accesorio imprescindible para llevar contigo a todas partes, ya sea al trabajo, al gimnasio o a tus aventuras al aire libre. ¡Haz un cambio positivo en el mundo mientras te mantienes hidratado con la botella de agua ecológica!




---



## ChatPromptTemplate
Enfocado en la creación de prompts para modelos en formato de chat, como GPT-4 en modo conversacional.

In [28]:
from langchain_openai import ChatOpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# Crear las plantillas de mensajes
system_message = SystemMessagePromptTemplate.from_template(
    "Eres un asistente que genera una descripción de un producto de forma convincente y breve."
)
human_message = HumanMessagePromptTemplate.from_template("{product_name}")

# Crear el prompt de chat
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

# Instanciar el modelo de chat
llm = ChatOpenAI(temperature=0.5)

# Crear la cadena
chain = LLMChain(llm=llm, prompt=chat_prompt)

# Ejecutar la cadena
input_data = {'product_name': 'Botella de agua ecológica'}
print(chain.run(input_data))

¡Lleva contigo esta botella de agua ecológica y ayuda a reducir el uso de plástico de un solo uso! Fabricada con materiales reciclables y reutilizables, esta botella es perfecta para mantener tu hidratación en todo momento. Su diseño moderno y funcional la convierte en el complemento ideal para tu estilo de vida sostenible. ¡Di no al plástico y sí a esta botella de agua ecológica!


### Cadenas de secuencia simple

SimpleSequentialChain

In [37]:
from langchain.chains import SimpleSequentialChain

llm = ChatOpenAI(temperature=0.5)
#cadena 01
first_template = ChatPromptTemplate.from_template("""
Indica si el texto denota una emocion en una sola palabra (positiva o Negativa): {mensaje}""")
chain_one = LLMChain(llm=llm, prompt=first_template)

#cadena 02
second_template =  ChatPromptTemplate.from_template("""
Envia un mensaje de agradecimiento o Disculpa si el resultado es positivo o Negativo:  {emocion}
""")
chain_two = LLMChain(llm=llm, prompt=second_template)

#Secuencia lineal
cadena_lineal = SimpleSequentialChain(chains=[chain_one, chain_two])
resultado = cadena_lineal.run("Su producto se malogro al dia siguiente")

print(resultado)

Disculpa por el resultado negativo, seguiré trabajando para mejorar y espero poder brindarte un mejor servicio en el futuro. ¡Gracias por tu comprensión!


SequentialChain

In [80]:
from langchain.chains import SequentialChain

llm = ChatOpenAI(temperature=0.5)
# cadena 01 | input: reseña | output: spanish_rev
one_template = ChatPromptTemplate.from_template("""
Traducir la siguiente reseña en español {review}
""")
chain_one = LLMChain(llm=llm, prompt=one_template, output_key="spanish_rev")

# Cadena 02 | input spanish_rev | output resumen
two_template = ChatPromptTemplate.from_template("""
resume la siguiente reseña en una frase {spanish_rev}
""")
chain_two = LLMChain(llm=llm, prompt=two_template, output_key="resumen")

#cadena 03 | input review | output lenguaje
three_template = ChatPromptTemplate.from_template("""
En que idioma se encuentra la siguiente reseña {review}
""")
chain_three = LLMChain(llm=llm, prompt=three_template, output_key="lenguaje")

#cadena 04 | input resumen,lenguaje | output mensaje_seg
four_template = ChatPromptTemplate.from_template("""
Escribe una respuesta de seguimiento a el siguiente resumen en el idioma especificado {resumen}, idioma: {lenguaje}
""")
chain_four = LLMChain(llm=llm, prompt=four_template, output_key="mensaje_seg")

#flujo de cadena

cadena_secuencial = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["spanish_rev", "resumen", "lenguaje", "mensaje_seg"]
)

In [81]:
review = """For all their convenience, Bluetooth headphones and earbuds have fundamental problems.
            Take their batteries (please). They're only fully rechargeable 300–500 times,
            which means that after just two or three years of moderate-to-heavy use,
            most people toss their depleted wireless ear-fi in a drawer and buy a new pair."""

In [82]:
result = cadena_secuencial.invoke(review)

In [83]:
print("Traduccion: "+ result['spanish_rev'] +"\n\n"+
      "Resumen: "+ result['resumen'] +"\n\n"+
      "Idioma: "+ result['lenguaje'] +"\n\n"+
      "Respuesta: "+ result['mensaje_seg']
      )

Traduccion: A pesar de toda su conveniencia, los auriculares y audífonos Bluetooth tienen problemas fundamentales. Tome sus baterías (por favor). Solo se pueden recargar completamente de 300 a 500 veces, lo que significa que después de solo dos o tres años de uso moderado a intenso, la mayoría de las personas tiran sus auriculares inalámbricos agotados en un cajón y compran un par nuevo.

Resumen: Los auriculares y audífonos Bluetooth tienen problemas de durabilidad debido a la limitada vida útil de sus baterías.

Idioma: La reseña está en inglés.

Respuesta: Thank you for bringing up the issue of durability with Bluetooth headphones and earphones. It is true that the limited lifespan of their batteries can be a concern for many users. Manufacturers are constantly working on improving battery technology to address this issue. In the meantime, it's important for consumers to properly care for their devices and follow recommended charging practices to maximize the lifespan of their batte

Cadena Condicional

In [61]:
llm = ChatOpenAI(temperature=0.5)

rw="Tengo un problema con los audifonos que me vendieron, no funcionan desde el primer dia"

first_prompt = ChatPromptTemplate.from_template(
    """analiza el sentimiento de la siguiente reseña
    y devuelve el resultado en una sola palabra, positivo o negativo {review}"""
    )

second_prompt = ChatPromptTemplate.from_template(
    """Genera una respuesta de agradecimiento para la siguiente reseña positiva {review}""")

third_prompt = ChatPromptTemplate.from_template(
    """Genera una respuesta de disculpas para la siguiente reseña negativa {review}""")

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key= "sentiment")
result_one = chain_one.run({"review": rw})

#cadena de respuesta condicional

if "positivo" in result_one.lower():
  chain_three = LLMChain(llm=llm, prompt=second_prompt,
                     output_key= "response")
elif "negativo" in result_one.lower():
  chain_three = LLMChain(llm=llm, prompt=third_prompt,
                     output_key= "response")
  result_three = chain_three.run({"review": rw})

  print(result_three)

Lamentamos mucho escuchar que has tenido problemas con los audífonos que adquiriste. Nos disculpamos por cualquier inconveniente que esto te haya causado. Por favor, contáctanos para que podamos resolver este problema de la mejor manera posible y encontrar una solución que te satisfaga. Tu satisfacción es nuestra prioridad y estamos comprometidos en brindarte el mejor servicio posible. ¡Gracias por tu comprensión!


Cadena Condicional como funcion

In [78]:
def main_chain(review, client):

  first_prompt = ChatPromptTemplate.from_template(
    """analiza el sentimiento de la siguiente reseña
    y devuelve el resultado en una sola palabra, positivo o negativo {review}"""
    )

  second_prompt = ChatPromptTemplate.from_template(
    """Genera una respuesta de agradecimiento para {client} en formato de correo para la siguiente reseña positiva {review}""")

  third_prompt = ChatPromptTemplate.from_template(
    """Genera una respuesta de disculpas para {client} en formato de correo para la siguiente reseña negativa {review}""")

  #cadena 1 analisis de sentimiento
  chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key= "sentiment")
  result_one = chain_one.run({"review": rw})

  #cadena 3 condicional
  if "positivo" in result_one.lower():
    chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key= "response")
  elif "negativo" in result_one.lower():
    chain_two = LLMChain(llm=llm, prompt=third_prompt,
                     output_key= "response")
    result_chain = chain_two.run({"review": rw, "client": client})
  return result_chain

In [79]:
review = """For all their convenience, Bluetooth headphones and earbuds have fundamental problems.
            Take their batteries (please). They're only fully rechargeable 300–500 times,
            which means that after just two or three years of moderate-to-heavy use,
            most people toss their depleted wireless ear-fi in a drawer and buy a new pair."""
client= "Javier Orihuela"

result = main_chain(review,client)
print(result)

Estimado Javier Orihuela,

Lamentamos profundamente la experiencia negativa que has tenido con los audífonos que adquiriste en nuestra tienda. Nos disculpamos sinceramente por cualquier inconveniente que esto te haya causado.

Queremos asegurarte que la satisfacción de nuestros clientes es nuestra prioridad y nos tomamos muy en serio cualquier problema que pueda surgir con nuestros productos. Por favor, háznoslo saber y estaremos encantados de ofrecerte una solución inmediata.

Por favor, acércate a nuestra tienda con los audífonos y el comprobante de compra para que podamos evaluar la situación y encontrar la mejor manera de resolver este inconveniente.

Una vez más, te pedimos disculpas por los inconvenientes causados y agradecemos tu comprensión y paciencia en este asunto.

Atentamente,
[Nombre de la empresa]
